In [ ]:
import numpy as np
from pandas import *
class MyLogReg():
  def __init__(self,weigths=None,n_iter=100,learning_rate=None,metric=None,y_pred=None,reg=None,l1_coef=0,l2_coef=0,sgd_sample=None,random_state=42):
    self.n_iter=n_iter
    self.learning_rate=learning_rate
    self.weights=weigths
    self.metric=metric
    self.y_pred=y_pred
    self.reg=reg
    self.l1_coef=l1_coef
    self.l2_coef=l2_coef
    self.sgd_sample=sgd_sample
    self.random_state=random_state
  def __str__(self):
    return "MyLineReg class: "+ ", ".join(('{}={}'.format(item, self.__dict__[item]) for item in self.__dict__))
  def __repr__(self):
     return "MyLineReg class: "+ ", ".join(('{}={}'.format(item, self.__dict__[item]) for item in self.__dict__))

  def fit(self,X,y,verbose=False):
    random.seed(self.random_state)
    self.weights=np.ones(X.shape[1]+1)
    eps = 1e-15
    X.insert(0,"ones",np.ones(X.shape[0]))
    is_double=isinstance(self.learning_rate, float)
    X.reset_index(drop=True,inplace=True)
    y.reset_index(drop=True,inplace=True)
    for i in range(1,self.n_iter+1):
      if isinstance(self.sgd_sample,int):
        sample_rows_idx = random.sample(range(X.shape[0]), self.sgd_sample)
      elif self.sgd_sample != None:
        sample_rows_idx = random.sample(range(X.shape[0]), int(self.sgd_sample*len(y)))
      else:
        sample_rows_idx=X.index

      X_cut = X.iloc[sample_rows_idx]
      y_cut = y.iloc[sample_rows_idx]
      y_pred = 1/(1+np.exp(-self.weights @ X_cut.T))
      if self.reg == "l1":
        logloss = -np.mean(y*np.log(y_pred+eps)+(1-y)*np.log(1-y_pred+eps)) + self.l1_coef * np.sum(self.weights)
        gradient =  (y_pred-y_cut)@X_cut/len(y_cut) + self.l1_coef * np.sign(self.weights)
      elif self.reg=="l2":
        logLoss = -np.mean(y*np.log(y_pred+eps)+(1-y)*np.log(1-y_pred+eps)) + self.l2_coef * np.sum(self.weights**2)
        gradient =  (y_pred-y_cut)@X_cut/len(y_cut) +  self.l2_coef * 2 * self.weights
      elif self.reg=="elasticnet":
        logLoss = -np.mean(y*np.log(y_pred+eps)+(1-y)*np.log(1-y_pred+eps)) +  self.l1_coef * np.sum(self.weights) + self.l2_coef * np.sum(self.weights**2)
        gradient =  (y_pred-y_cut)@X_cut/len(y_cut) + self.l1_coef * np.sign(self.weights)+ self.l2_coef * 2 * self.weights
      else:
        logLoss = -np.mean(y*np.log(y_pred+eps)+(1-y)*np.log(1-y_pred+eps))
        gradient =  (y_pred-y_cut)@X_cut/len(y_cut)

      if not is_double:
        self.weights-=self.learning_rate(i) * gradient
      else:
        self.weights-=self.learning_rate * gradient

      if verbose and (i+1)%verbose==0:
        if self.metric!=None:
          print(f"{i+1} | loss : {logloss: .2f}")
        else:
           print(f"{i+1} | loss : {logloss: .2f}")
    self.y_pred=1/(1+np.exp(-self.weights @ X.T))

  def get_coef(self):
    return self.weights[1:]

  def predict_proba(self,X):
    return np.mean(1/(1+np.exp(-self.weights @ X.T)))
  def predict(self,X):
    X.insert(0,"ones",np.ones(X.shape[0]))
    return np.sum((1/(1+np.exp(-self.weights @ X.T)) > 0.5).astype(int))

  def get_best_score(self):
    y_pred = self.y_pred
    tp = np.sum(((self.y_pred > 0.5).astype(int) == 1) & (y == 1))
    tn = np.sum(((self.y_pred > 0.5).astype(int) == 0) & (y == 0))
    fp = np.sum(((self.y_pred > 0.5).astype(int) == 1) & (y == 0))
    fn = np.sum(((self.y_pred > 0.5).astype(int) == 0) & (y == 1))
    if self.metric=="accuracy":
      return (tp+tn)/(tp+tn+fp+fn)
    elif self.metric == 'precision':
      return tp/(tp+fp)
    elif self.metric=="recall":
       return tp/(tp+fn)
    elif self.metric =="f1":
      pr=tp/(tp+fp)
      rc=tp/(tp+fn)
      return 2*pr*rc/(pr+rc)
    elif self.metric=="roc_auc":
        y_score = y_pred.round(10)
        df = concat([y_score, y], axis=1)
        df = df.sort_values(by=0, ascending=False)

        positives = df[df[1] == 1]
        negatives = df[df[1] == 0]

        total = 0
        for _, row in negatives.iterrows():
            score_higher = (positives[0] > row[0]).sum()
            score_equal = (positives[0] == row[0]).sum()
            total += score_higher + 0.5 * score_equal

        return total / (positives.shape[0] * negatives.shape[0])


a=MyLogReg(learning_rate=2.0,metric='roc_auc')
y=Series([1,0,0,1,1])
a.fit(DataFrame(np.random.randn(5, 3)),y)
a.get_coef()
a.get_best_score()

1.0

In [ ]:
y_pred=[0.91,0.86,0.78,0.6,0.6,0.55,0.51,0.46,0.45,0.45,0.42]
y_sorted = np.sort(y_pred)[::-1]
y_class = [1,0,0,1,0,1,0,0,0,1,0]
p = np.sum(y_class)
n= len(y_class)-p

sum = 0
q=len(y_sorted)

for i in range(q):

  for j in range(q):
    sum1=0
    sum2=0
    if y_class[i]<y_class[j]:
      sum1+=1
      if y_sorted[i]==y_sorted[j]:
          sum2+=0.5
      elif y_sorted[i]<y_sorted[j]:
          sum2+=1
    sum+=sum1*sum2


print(sum / (p * n))

0.6071428571428571


In [ ]:
import pandas as pd
y_pred=Series([0.91,0.86,0.78,0.6,0.6,0.55,0.51,0.46,0.45,0.45,0.42])
y_score = y_pred.round(10)
y_true = Series([1,0,0,1,0,1,0,0,0,1,0])
df = pd.concat([y_score, y_true], axis=1)
df = df.sort_values(by=0, ascending=False)

positives = df[df[1] == 1]
negatives = df[df[1] == 0]

total = 0
for _, row in negatives.iterrows():
    score_higher = (positives[0] > row[0]).sum()
    score_equal = (positives[0] == row[0]).sum()
    total += score_higher + 0.5 * score_equal

print( total / (positives.shape[0] * negatives.shape[0]))


0.6071428571428571
